In [1]:
import numpy as np
import pandas as pd
import time
from datetime import datetime,timedelta
import json
import re

In [2]:
#根据log_index，增量读取log内容
def load_log(file_name, log_index):
    log_value_list = list()
    date_list = list()
    status_list = list()
    try:
        with open(file_name,'rb') as f:
            for index,content in enumerate(f):
                if index == log_index:
                    log_data = json.loads(content.decode('utf8'))
                    log_data['@timestamp'] = str(datetime.strptime(log_data['@timestamp'][0:19],'%Y-%m-%dT%H:%M:%S'))
                    log_value_list.append(log_data)
                    date_list.append(log_data['@timestamp'])
                    log_index += 1
                    if log_data['status'] not in status_list:
                        status_list.append(log_data['status'])
    except Exception as e:
        print(e)
    #将文本内容转化为pandas的dataframe
    #temp_log = pd.DataFrame(log_value_list)
    print(status_list)
    return  log_value_list,log_index

In [3]:
time_step = 60 #每x秒读取一次log文件
time_windows = 120 #秒,时间窗口
ip_dict = dict() #ip与对应操作在矩阵的下标
ip_reserve_list = list() #操作矩阵下标对应的ip
all_matrix = np.empty([0, 6],dtype=int)  #操作矩阵
all_log = pd.DataFrame() #所有的日志数据
temp_log_data = pd.DataFrame() #某时间范围的日志数据
all_url_list = list()
time_index = '2017-07-10 06:25:08' #记录当前的时间
log_index = 0 #记录读到第几行了
ip_behavior_dict = dict() #记录每个ip每分钟请求的页面url

increment_log_data, log_index = load_log('0713_access_sdsl_shunnengnet_com_json.log', 0)

['200', '302', '304', '403', '499', '404', '503', '408', '400', '504']


In [4]:
#截取url，保留其访问传参方式（POST/GET/PUT/DELETE）,保留url的头与尾
def deal_url(url):
    url_str = url.replace(' HTTP/1.1','')
    if url_str.find(' ') > -1:
        url_type = url_str[0 : url_str.find(' ')+1]
    if url_str.find('/') > -1 and url_str.find('?') > -1:
        url_str = url_type + url_str[url_str.find('/')+1 : url_str.find('?')]
    if url_str.find('/') > -1 and url_str.rfind('/') > -1:
        url_str = url_str[0 : url_str.find('/')] + '/.../' + url_str[url_str.rfind('/')+1 : ]
    return url_str

In [5]:
#根据日志内容，创建一个信息的op_detail
def create_op_detail(row_log):
    op_detail = dict()
    op_detail['start_time'] = row_log['@timestamp']
    op_detail['end_time'] = row_log['@timestamp']
    op_detail['urls'] = list()
    op_detail['urls'].append(deal_url(row_log['request']))
    return op_detail

In [6]:
#传入开始结束时间的字符串(yyyy-MM-dd HH:mm:ss)，计算时间差并返回时差(秒)
def cacl_time_difference(start_time_str, end_time_str):
    start_time = datetime.strptime(start_time_str,'%Y-%m-%d %H:%M:%S')
    end_time = datetime.strptime(end_time_str,'%Y-%m-%d %H:%M:%S')
    return int((end_time - start_time).seconds)

In [7]:
#读取log，生成携带用户行为的复合型字典
def create_user_behavior_dict(increment_log_data, user_behavior_dict):
    
    for x in increment_log_data:
        if x['sip'] != '':
            
            if x['sip'] not in user_behavior_dict.keys():
                user_behavior_dict[x['sip']] = list()

            if len(user_behavior_dict[x['sip']]) == 0: #opreation内没有op_detail,则创建op_detail
                user_behavior_dict[x['sip']].append(create_op_detail(x))

            if len(user_behavior_dict[x['sip']]) > 0: #opreation内已存在op_detail，此时需要循环该list的所有dict，比较时间差，如果两者时间相差<120s，更新结束时间，添加url；否则继续遍历比较直至找不到合适的区间为止，若找不到合适区间，新建
                new_opreation = 1
                length = len(user_behavior_dict[x['sip']])
                i = 0
                for op_detail in user_behavior_dict[x['sip']]:
                    time_difference = cacl_time_difference(op_detail['end_time'], x['@timestamp'])
                    if time_difference < 120:
                        op_detail['end_time'] = x['@timestamp']
                        op_detail['urls'].append(deal_url(x['request']))
                        new_opreation = 0
                        break
                    if new_opreation == 1 and i+1 == length:
                        user_behavior_dict[x['sip']].append(create_op_detail(x))
                        break
                        
                    i += 1
                    
                
    return user_behavior_dict
        
        

In [8]:
user_behavior_dict = dict()

In [9]:
start_time = datetime.now()
user_behavior_dict = create_user_behavior_dict(increment_log_data, user_behavior_dict)
end_time = datetime.now()
print((end_time - start_time).seconds)

25


In [10]:
#定义模板list
templates = list()
templates.append(['GET /common/.../notice.do','POST /common/.../chooseHospital.do','GET /common/.../chooseExpert.do','GET /common/.../deptList.do','POST /.../getDept.do','GET /common/.../scheduleByDept.do','POST /.../scheduleByDept.do','GET /common/.../arrangementList.do','POST /.../getTimeList.do','GET /common/.../reserveConfirm.do','POST /.../reserveConfirm.do','POST /.../reserve.do']) #普通专家预约
templates.append(['GET /common/.../notice.do','POST /common/.../chooseHospital.do','GET /common/.../chooseExpert.do','GET /common/.../expertNotice.do','POST /common/.../deptList.do','POST /.../getDept.do','GET /common/.../scheduleByDept.do','POST /.../scheduleByDept.do','GET /common/.../arrangementList.do','POST /.../getTimeList.do','GET /common/.../reserveConfirm.do','POST /.../reserveConfirm.do','POST /.../reserve.do'])#知名专家预约
templates.append(['GET /common/.../notice.do','POST /common/.../chooseHospital.do','GET /common/.../chooseExpert.do','GET /common/.../specialClinic.do','POST /.../getDepts.do','GET /common/.../specialSchedule.do','POST /.../specialSchedule.do','GET /common/.../arrangementList.do','POST /.../getTimeList.do','GET /common/.../reserveConfirm.do','POST /.../reserveConfirm.do','POST /.../reserve.do'])#特色门诊预约
templates.append(['GET /common/.../patientinfo.do','GET /common/.../reserveList.do','POST /.../getReserveList.do','GET /common/.../reserveDetail.do','POST /.../cancelReserve.do'])#取消预约
templates.append(['GET /common/.../patientinfo.do','GET /common/.../patientMgr.do ','GET /.../patientinfo.do','GET /common/.../addPatient.do','POST /.../checkIdCard.do','POST /.../create.do'])#新增就诊人
templates.append(['GET /common/.../patientinfo.do','GET /common/.../patientMgr.do ','GET /common/.../editPatient.do','POST /.../getPatientInfoById.do','POST /.../setDefault.do',''])#设置默认就诊人
templates.append(['GET /common/.../patientinfo.do','GET /common/.../patientMgr.do ','GET /common/.../editPatient.do','POST /.../getPatientInfoById.do','GET /common/.../bind.do','POST /.../bindHosCard.do'])#绑卡
templates.append(['GET /common/.../patientinfo.do','GET /common/.../patientMgr.do ','GET /common/.../editPatient.do','POST /.../getPatientInfoById.do','GET /.../deletePatient.do'])#删除就诊人
templates.append(['GET /.../init.do','GET /common/.../reportType.do ','GET /common/.../reportList.do','GET /.../getReportList.do','GET /common/.../reportDetail.do','GET /.../getReportDeatil.do'])#门诊报告
templates.append(['GET /.../init.do','GET /common/.../reportType.do ','GET /common/.../reportList.do','GET /.../getReportInList.do','GET /.../getReportDeatil.do'])#住院报告
templates.append(['GET /.../init.do','GET /common/.../index.do','GET /common/.../balance.do','GET /.../patientBalance.do'])#门诊预交金
templates.append(['GET /.../init.do','GET /common/.../index.do','GET /common/.../hosBalance.do','POST /.../inBalance.do'])#住院预交金
templates.append(['GET /.../init.do','GET /common/.../index.do','GET /common/.../outpFeeList.do','POST /.../feeList.do','GET /common/.../feeDetail.do','POST /.../feeDetail.do'])#门诊费用
templates.append(['GET /.../init.do','GET /common/.../index.do','GET /common/.../inpFeeList.do','POST /.../feeList.do','GET /common/.../feeDetail.do','POST /.../feeDetail.do'])#住院费用
templates.append(['GET /.../init.do','GET /common/.../reserveQueueList.do','POST /.../getQueueList.do'])#候诊队列
templates.append(['POST coreServlet'])#处理微信发送来的数据
templates.append(['GET common/.../arrangementList.do', 'GET /.../404.do','POST /.../getTimeList.do'])#异常模板1
#templates.append(['POST /.../getTimeList.do','POST /.../getTimeList.do','GET common/.../arrangementList.do','GET /.../404.do','GET common/.../arrangementList.do','GET /.../404.do'])#异常模板1
templates.append(['GET common/.../scheduleByDept.do','POST /.../scheduleByDept.do','POST /.../getTimeList.do','GET common/.../arrangementList.do','POST /.../getTimeList.do', 'POST /.../getTimeList.do','POST /.../getTimeList.do','POST /.../getTimeList.do', 'POST /.../getTimeList.do'])#异常模板2
#templates.append(['GET common/.../arrangementList.do', 'GET common/.../arrangementList.do', 'GET common/.../arrangementList.do', 'POST /.../getTimeList.do', 'POST /.../getTimeList.do', 'POST /.../getTimeList.do', 'POST /.../getTimeList.do', 'POST /.../getTimeList.do', 'POST /.../getTimeList.do', 'POST /.../getTimeList.do', 'POST /.../getTimeList.do', 'POST /.../getTimeList.do'])#异常模板3
templates.append(['GET /common/.../deptList.do','POST /.../getDept.do','GET /common/.../scheduleByDept.do','POST /.../scheduleByDept.do','GET /common/.../arrangementList.do','POST /.../getTimeList.do'])

In [149]:
probability_dict = dict()

for x in user_behavior_dict.keys():
    for op_detail in user_behavior_dict[x]:
        matched_operations = np.zeros(len(templates),dtype='float32')
        total_secs = cacl_time_difference(op_detail['start_time'], op_detail['end_time']) + 5
        op_per_sec = len(op_detail['urls'])/total_secs
        
        for idx,template in enumerate(templates):
            n_template = len(template)
            count = 0
            probability_count = 0
            for str in template:
                if str in op_detail['urls']:
                    probability_count += 1
            for url in op_detail['urls']:
                if url in template:
                    count += 1
            n_op = max(len(op_detail['urls']),16)

            #(probability_count / n_template) 模板被匹配到的百分比
            #(count / n_op) 刷单操作占总操作的百分比，刷的越厉害这个越高
            # op_per_sec 平均每秒中的操作数，操作越快越像刷单
            matched_operations[idx] = (probability_count / n_template) * (count / n_op) #* op_per_sec
        if x not in probability_dict.keys():
            probability_dict[x] = matched_operations.reshape([1,19])
        else:
            probability_dict[x] = np.vstack((probability_dict[x],matched_operations))
        
        

In [133]:
idx = 0
for x in user_behavior_dict.keys():
    idx += len(user_behavior_dict[x])
idx

18077

In [134]:
len(probability_dict['223.96.152.23'])

8

In [103]:
frequency_dict = dict()
for x in increment_log_data:
    if x['sip'] != '':
        if x['sip'] not in frequency_dict.keys():
            frequency_dict[x['sip']] = 0
        frequency_dict[x['sip']] += 1

In [104]:
frequency_list = sorted(frequency_dict.items(), key=lambda d: d[1])

In [105]:
#查看大于某频度值的ip
# for x in frequency_list:
#     if x[1] > 500:
#         for y in range(0, len(user_behavior_dict[x[0]])):
#             print(x[0])
#             print(user_behavior_dict[x[0]][y])
#             print(probability_dict[x[0]][y])

In [106]:
import time
#速度变慢的罪魁祸首
def judge_time_range(start_time_str, end_time_str, time_str):
    start_date = time.strptime(start_time_str,'%Y-%m-%d %H:%M:%S')
    end_date = time.strptime(end_time_str,'%Y-%m-%d %H:%M:%S')
    time_date = time.strptime(time_str,'%Y-%m-%d %H:%M:%S')
    
    if time_date > start_date and time_date < end_date:
        return True
    else:
        return False
    

In [107]:
def get_operation_label(ip, time):
    global probability_dict
    class_type = 0
    for x in user_behavior_dict:
        if x == ip:
            index = 0
            for op_detail in user_behavior_dict[x]:
                start_str = op_detail['start_time']
                end_str = op_detail['end_time']
                if judge_time_range(start_str, end_str, time):
                    class_type = probability_dict[ip][-,16:].sum()
                    #if probability_dict[ip][:,16:].sum() > 1:
                        #class_type = 1
                    #elif probability_dict[ip][index,17] > 0.4:
                        #class_type = 1
                   # elif probability_dict[ip][index,18] > 0.4:
                        #class_type = 1
                    break #如果匹配到了，也给了class_type了，还继续循环干什么
                else:
                    index += 1
    return class_type

SyntaxError: invalid syntax (<ipython-input-107-363f17e16b28>, line 11)

In [108]:
probability_dict['27.204.15.164'].shape

(1, 19)

In [109]:
probability_dict['39.71.57.17'][:,16:].sum()

0.61949319

In [110]:
get_operation_label('27.204.15.164', '2017-07-13 19:18:56')

4.5126047

In [111]:
#----------------------start
class IPAccessInfo(object):
    def __init__(self, all_url_list, all_url_dict_list):
        self.all_url_list = all_url_list
        self.all_url_dict_list = all_url_dict_list
        
import pickle
fp = open('url_info.pkl','rb',True)
iPAccessInfo = pickle.load(fp)

In [112]:
from keras.models import Sequential
from keras.layers import Embedding

#列出唯一的url
url_dict = dict()
for urls in iPAccessInfo.all_url_list:
    for url in urls:
        if url not in url_dict.keys():
            url_dict[url] = 1
        else:
            url_dict[url] = url_dict[url] + 1
            
url_list = list(url_dict.keys())

#将url embedding，转为固定长度的向量
num_urls = len(url_list)
model = Sequential()
model.add(Embedding(num_urls, 32, input_length=1))
model.compile('rmsprop', 'mse')

input_array = np.array(range(num_urls))   #生成一个[num_url,1]的矩阵
output_array = model.predict(input_array)
print (output_array.shape)
print (output_array)

# 建立url_dict备查
url_dict = dict()
idx = 0
for url in url_list:
    url_dict[url] = output_array[idx,0,:]
    idx = idx + 1

(162, 1, 32)
[[[ 0.03654477 -0.03177589 -0.02952494 ...,  0.0237189   0.00231814
    0.03386502]]

 [[-0.03017677 -0.00110988  0.0442226  ..., -0.0302777  -0.01121916
    0.01873815]]

 [[ 0.04072422 -0.02663457  0.02623384 ..., -0.01469091 -0.01961044
    0.00813953]]

 ..., 
 [[ 0.0428668  -0.04194156 -0.01407099 ...,  0.02149953  0.0487067
    0.02792516]]

 [[ 0.02159942 -0.02935338  0.02947632 ..., -0.0206544   0.01012788
    0.02063049]]

 [[-0.00404527  0.0194104  -0.03394111 ..., -0.01592012  0.00888164
    0.02377509]]]


In [135]:
#老版本数据转换，将按照时间窗口、ip进行归类的urls列表，转化为时序data
# def operations_to_sequence(all_url_dict_list):
#     operation_matrix = np.zeros((len(all_url_dict_list),20,32))
#     ip_operation_index = 0
#     for ip_operations in all_url_dict_list: #对于每小段窗口，以ip归类的urls集合
#         ip_operation_url_index = 0
#         for url in ip_operations['urls']: #如果是dict字典的话
#             url_vector = url_dict[url]
#             operation_matrix[ip_operation_index, ip_operation_url_index, :] = url_vector
#             ip_operation_url_index = ip_operation_url_index + 1
#             if ip_operation_url_index>=20:  #不超过20个长度的序列，可以改
#                 break
#         ip_operation_index = ip_operation_index + 1
#     return operation_matrix


#新版本数据转换，将按照ip地址，然后里面的几段行为数据，转化为时序data
# 数据格式为：
# {'223.104.186.241': [{'end_time': '2017-07-13 06:25:32',
#    'start_time': '2017-07-13 06:25:02',
#    'urls': ['POST /.../getDept.do',
#     'POST /.../getDept.do',
#     'GET common/.../scheduleByDept.do',
#     'POST /.../scheduleByDept.do',
#     'GET common/.../arrangementList.do',
#     'GET common/.../arrangementList.do',
#     'POST /.../getTimeList.do',
#     'GET common/.../reserveConfirm.do',
#     'POST /.../reserveConfirm.do',
#     'GET common/.../patientList.do',
#     'GET /.../patientinfo.do',
#     'GET common/.../reserveConfirm.do',
#     'POST /.../reserveConfirm.do',
#     'POST /.../reserve.do']},
#   {'end_time': '2017-07-13 07:31:18',
#    'start_time': '2017-07-13 07:26:36',
#    'urls': ['GET /.../noticePage.html',

def operations_to_sequence(user_behavior_dict):
    #计算这个dict中有多少段操作
    n_op_details = 0
    for x in user_behavior_dict.keys():
        n_op_details += len(user_behavior_dict[x])
    
    operation_matrix = np.zeros((n_op_details,30,32))
    ip_operation_index = 0
    for ip in user_behavior_dict.keys(): #对于每个ip,得到op_detail列表
        for op_detail in user_behavior_dict[ip]: #对于每一组urls操作
            ip_operation_url_index = 0
            for url in op_detail['urls']: #对于每一个url
                url_vector = url_dict[url]  #翻译为url对应的vector
                operation_matrix[ip_operation_index, ip_operation_url_index, :] = url_vector
                ip_operation_url_index += 1
                if ip_operation_url_index>=30:  #不超过20个长度的序列，可以改
                    break
            ip_operation_index += 1
    return operation_matrix

In [136]:
data = operations_to_sequence(user_behavior_dict)
print (data.shape) #数据编码，我就是快，数据量大640倍，还是快。。。

(18077, 30, 32)


In [140]:
n_class = 2
# 旧版的数据转换
# def label_operations(all_url_dict_list):
#     operation_label = np.zeros(len(all_url_dict_list)) # 先按照0 1 2 3，一维数组排序
#     ip_operation_index = 0
#     for ip_operations in all_url_dict_list: #对于每小段窗口，以ip归类的urls集合
#         operation_label[ip_operation_index] = get_operation_label(ip_operations['ip'],ip_operations['time'])
#         ip_operation_index = ip_operation_index + 1
        
#     #转化为one-hot编码的label
#     return operation_label


def label_operations(user_behavior_dict):
    #计算这个dict中有多少段操作
    n_op_details = 0
    for x in user_behavior_dict.keys():
        n_op_details += len(user_behavior_dict[x])
        
    operation_labels = np.zeros(n_op_details) # 先按照0 1 2 3，一维数组排序
    ip_operation_index = 0
    for ip in user_behavior_dict.keys(): #对于每个ip,得到op_detail列表
        for op_index in range(len(user_behavior_dict[ip])): #对于每一组urls操作，得到其index 
            prob = probability_dict[ip][op_index,16:].sum() #每一小组op_detail['urls']对应的向量，后三列相加
            if prob>0.2:
                operation_labels[ip_operation_index] = 1
            else:
                operation_labels[ip_operation_index] = 0
            ip_operation_index = ip_operation_index + 1
        
    return operation_labels

In [141]:
# tmd为啥这么慢，为啥这么慢，为啥这么慢啊!!!!!!
labels = label_operations(user_behavior_dict)
print (labels.shape)

(18077,)


In [142]:
(labels==1).sum()

380

In [124]:
# #labels
# b = np.argsort(labels)
# b = b.tolist()
# b.reverse()
# for x in b:
#     if labels[x] > 0.9 and labels[x] < 1:
#         #ip = iPAccessInfo.all_url_dict_list[x]['ip']
#         print(iPAccessInfo.all_url_dict_list[x])
        
        

In [143]:
(labels==1).sum() #验证每个类别的个数

380

In [111]:
#网址embedding表示，应该不需要进行正规化吧
# from sklearn.preprocessing import MinMaxScaler
# scaler = MinMaxScaler()

# #由于时序序列数据，是（？，时间长度，数据长度）的，所以应该执行一个reshape
# X = scaler.fit_transform(data.reshape([-1,20*32]))
# X = X.reshape([-1,20,32])

In [60]:
# 定义保存数据的类
import numpy as np

#定义类
class IPAccessData(object):
    def __init__(self, X, y):
        self.X = X
        self.y = y
#         self.ip_dict = ip_dict
#         self.ip_reserve_list = ip_reserve_list
#         self.data_scaler = data_scaler

# 保存文件为pkl
import pickle

iPAccessData = IPAccessData(data,labels)
fp = open('iPAccessData.pkl','wb',True)
pickle.dump(iPAccessData, fp)

In [144]:
ip_labels = np.zeros(len(user_behavior_dict.keys()))
ips = list(user_behavior_dict.keys())
ip_index = 0
op_index = 0
for x in user_behavior_dict.keys():
    n_operations = len(user_behavior_dict[x])
    ip_labels[ip_index] = labels[op_index:op_index+n_operations].sum()/n_operations
    ip_index += 1
    op_index += n_operations


In [146]:
[ips[idx] for idx,x in enumerate(ip_labels) if x>=0.6]

['223.81.196.208',
 '112.225.44.216',
 '112.232.150.64',
 '112.224.19.61',
 '39.64.194.29',
 '112.231.229.204',
 '171.127.188.210',
 '27.204.4.166',
 '39.64.155.14',
 '218.201.124.231',
 '112.224.2.101',
 '112.224.65.236',
 '113.128.56.35',
 '123.134.182.6',
 '111.16.227.74',
 '124.128.251.67',
 '117.136.94.124',
 '111.37.0.113',
 '112.232.125.242',
 '218.56.242.97',
 '123.134.230.0',
 '144.52.185.248',
 '112.229.24.207',
 '123.134.153.181',
 '60.216.156.84',
 '123.134.255.102',
 '144.12.24.42',
 '112.224.1.112',
 '112.232.15.90',
 '123.134.225.75',
 '27.204.19.242',
 '223.96.152.38',
 '123.233.53.78',
 '119.176.250.17',
 '112.229.247.84',
 '124.128.121.17',
 '39.70.4.161',
 '119.162.49.146',
 '112.234.91.162',
 '144.255.5.41',
 '39.64.179.237',
 '120.192.66.41',
 '119.163.6.59',
 '27.211.35.192',
 '112.224.19.2',
 '58.56.92.14',
 '123.233.150.157',
 '218.58.175.182',
 '61.156.219.212',
 '112.38.44.61',
 '123.232.148.60',
 '123.233.209.252',
 '124.133.236.92',
 '123.233.216.155',
 '112

In [150]:
ip = '119.162.49.146'
print (probability_dict[ip])
user_behavior_dict[ip]

[[ 0.046875    0.04326923  0.00520833  0.          0.          0.          0.
   0.          0.          0.          0.          0.          0.          0.
   0.          0.          0.125       0.3125      0.09375   ]]


[{'end_time': '2017-07-13 12:36:40',
  'start_time': '2017-07-13 12:36:26',
  'urls': ['GET common/.../notice.do',
   'GET common/.../notice.do',
   'POST common/.../chooseHospital.do',
   'GET common/.../chooseExpert.do',
   'GET common/.../deptList.do',
   'POST /.../getDept.do',
   'GET common/.../scheduleByDept.do',
   'POST /.../scheduleByDept.do',
   'GET common/.../arrangementList.do',
   'GET common/.../arrangementList.do',
   'POST /.../getTimeList.do']}]